## Numba: compiled Python

We know that Python is slow—that's why we use Python and C++. However, "rewrite in C++" is a roadblock in development from simple Python tests to a full-dataset algorithm.

It's not always necessary to rewrite in C++. There are compilers for Python.

Demonstration: start with a pure Python algorithm. The code below draws a fractal.

In [1]:
%matplotlib inline
import numpy, numba, matplotlib.pyplot, time

In [2]:
def run_python(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32) + maxiterations
    for h in range(height):
        for w in range(width):                  # for each pixel (h, w)...
            z = c[h, w]
            for i in range(maxiterations):      # iterate at most 20 times
                z = z**2 + c[h, w]              # applying z → z² + c
                if abs(z) > 2:                  # if it diverges (|z| > 2)
                    fractal[h, w] = i           # color the plane with the iteration number
                    break                       # we're done, no need to keep iterating
    return fractal

In [3]:
starttime = time.time()
fractal = run_python(800, 1200)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (800 * 1200)))

5781.266838312149 ns per pixel


In [36]:
fig, ax = matplotlib.pyplot.subplots(figsize=(18, 12))
ax.imshow(fractal)

Since Numpy operates array-at-a-time, you have to completely rethink the problem to use Numpy (same for Awkward).

In [5]:
def run_numpy(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32) + maxiterations
    z = c
    for i in range(maxiterations):
        z = z**2 + c                                            # applying z → z² + c
        diverged = numpy.absolute(z) > 2                        # |z| > 2 is "divergence"
        diverging_now = diverged & (fractal == maxiterations)   # some are already done
        fractal[diverging_now] = i                              # just set the new ones
        z[diverged] = 2                                         # clamp diverged at 2
    return fractal

In [6]:
starttime = time.time()
fractal = run_numpy(1600, 2400)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (1600 * 2400)))

379.4632852077484 ns per pixel


Numpy is 13× faster than pure Python _even though_ it can't `break` when done with a given pixel.

With Numba, we can make the pure Python code fast by inserting `@numba.jit` before the function.

In [7]:
@numba.jit
def run_numba(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32) + maxiterations
    for h in range(height):
        for w in range(width):                  # for each pixel (h, w)...
            z = c[h, w]
            for i in range(maxiterations):      # iterate at most 20 times
                z = z**2 + c[h, w]              # applying z → z² + c
                if abs(z) > 2:                  # if it diverges (|z| > 2)
                    fractal[h, w] = i           # color the plane with the iteration number
                    break                       # we're done, no need to keep iterating
    return fractal

In [8]:
starttime = time.time()
fractal = run_numba(3200, 4800)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (3200 * 4800)))

135.26005980869135 ns per pixel


Numba is 50× faster than pure Python without changing the code.

Converting it all to C++ is not an automatic win.

In [9]:
import ROOT
tmpname = "run_cpp1"
ROOT.gInterpreter.Declare("""
#include<complex>
void %s(int height, int width, int maxiterations, size_t c_ptr, size_t fractal_ptr) {
    double* c = (double*)(c_ptr);
    int* fractal = (int*)(fractal_ptr);
    for (int h = 0;  h < height;  h++) {
        for (int w = 0;  w < width;  w++) {
            double creal = c[2 * (h + height*w)];
            double cimag = c[2 * (h + height*w) + 1];
            std::complex<double> ci = std::complex<double>(creal, cimag);
            std::complex<double> z = ci;
            for (int i = 0;  i < maxiterations;  i++) {
                z = z*z + ci;
                if (std::abs(z) > 2) {
                    fractal[h + height*w] = i;
                    break;
} } } } }""" % tmpname)

def run_cpp(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.full(c.shape, maxiterations, dtype=numpy.int32)
    getattr(ROOT, tmpname)(height, width, maxiterations, c.ctypes.data, fractal.ctypes.data)
    return fractal

Welcome to JupyROOT 6.14/04


In [10]:
starttime = time.time()
fractal = run_cpp(3200, 4800)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (3200 * 4800)))

534.8232885201772 ns per pixel


This is only 10× faster than pure Python, whereas Numba was 50×. Much of the gap is due to the fact that ROOT currently compiles C++ in `Declare` without optimization (`-O0`).

To really do this right, make a separate file and compile it.

In [11]:
%%writefile tmp.cpp
#include<complex>
#include<pybind11/pybind11.h>

void run_cpp(int height, int width, int maxiterations, size_t c_ptr, size_t fractal_ptr) {
    double* c = (double*)(c_ptr);
    int* fractal = (int*)(fractal_ptr);
    for (int h = 0;  h < height;  h++) {
        for (int w = 0;  w < width;  w++) {
            double creal = c[2 * (h + height*w)];
            double cimag = c[2 * (h + height*w) + 1];
            std::complex<double> ci = std::complex<double>(creal, cimag);
            std::complex<double> z = ci;
            for (int i = 0;  i < maxiterations;  i++) {
                z = z*z + ci;
                if (std::abs(z) > 2) {
                    fractal[h + height*w] = i;
                    break;
                }
            }
        }
    }
}
PYBIND11_MODULE(tmp, m) {
    m.def("run_cpp", &run_cpp, "the inner loop");
}

Overwriting tmp.cpp


In [12]:
!c++ -O3 -ffast-math -Wall -shared -std=c++11 -fPIC `python -m pybind11 --includes` tmp.cpp -o tmp`python3-config --extension-suffix`

In [26]:
import tmp
starttime = time.time()
height, width, maxiterations = 3200, 4800, 20
y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
c = x + y*1j
fractal = numpy.full(c.shape, maxiterations, dtype=numpy.int32)
tmp.run_cpp(height, width, maxiterations, c.ctypes.data, fractal.ctypes.data)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (3200 * 4800)))

68.87235989173253 ns per pixel


With custom C++ code, I could get 85× faster than pure Python, but _only_ when using the `-ffast-math` flag, which sacrifices floating-point accuracy for speed.

Numba has a `fastmath=True` option, but the gain wasn't as significant (only 50× → 55×).

In [31]:
def run_numba(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32) + maxiterations
    return inner_loop(height, width, maxiterations, c, fractal)

@numba.jit(nopython=True, fastmath=True)        # Numba has a fastmath option, too...
def inner_loop(height, width, maxiterations, c, fractal):
    for h in range(height):
        for w in range(width):                  # for each pixel (h, w)...
            z = c[h, w]
            for i in range(maxiterations):      # iterate at most 20 times
                z = z**2 + c[h, w]              # applying z → z² + c
                if abs(z) > 2:                  # if it diverges (|z| > 2)
                    fractal[h, w] = i           # color the plane with the iteration number
                    break                       # we're done, no need to keep iterating
    return fractal

In [34]:
starttime = time.time()
fractal = run_numba(3200, 4800)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (3200 * 4800)))

109.99338701367378 ns per pixel


But Numba has more tricks: `range` → `numba.prange` parallelizes the loop, if possible (equivalent to OpenMP's "parallel for").

In [54]:
def run_numba(height, width, maxiterations=20):
    y, x = numpy.ogrid[-1:0:height*1j, -1.5:0:width*1j]
    c = x + y*1j
    fractal = numpy.zeros(c.shape, dtype=numpy.int32) + maxiterations
    return inner_loop(height, width, maxiterations, c, fractal)

@numba.jit(nopython=True, fastmath=True, parallel=True)
def inner_loop(height, width, maxiterations, c, fractal):
    for h in numba.prange(height):
        for w in numba.prange(width):           # for each pixel (h, w)...
            z = c[h, w]
            for i in range(maxiterations):      # iterate at most 20 times
                z = z**2 + c[h, w]              # applying z → z² + c
                if abs(z) > 2:                  # if it diverges (|z| > 2)
                    fractal[h, w] = i           # color the plane with the iteration number
                    break                       # we're done, no need to keep iterating
    return fractal

In [67]:
starttime = time.time()
fractal = run_numba(6400, 9600)
print("{0} ns per pixel".format(1e9 * (time.time() - starttime) / (6400 * 9600)))

29.719830490648746 ns per pixel


On my 12 core laptop, this is now 195× faster than the pure Python loop.